In [2]:
import numpy as np

# 무작위 데이터
x=np.random.rand(10,1,28,28)
x.shape

(10, 1, 28, 28)

In [4]:
x[0].shape
x[1].shape

(1, 28, 28)

In [6]:
x[0,0]

array([[5.95107326e-02, 2.23795261e-01, 5.80987021e-01, 8.09514783e-01,
        7.24144018e-01, 8.40046538e-01, 9.84099778e-01, 9.89853561e-01,
        6.52765688e-02, 4.06700286e-01, 2.53434180e-01, 8.77767543e-01,
        4.58562167e-01, 5.49207337e-01, 9.60510182e-01, 5.75332527e-01,
        7.39838483e-01, 5.71007079e-01, 8.55005979e-01, 7.63659090e-02,
        8.09548829e-01, 9.12755620e-02, 2.52512110e-01, 6.85537479e-01,
        7.75261462e-02, 6.93342435e-01, 6.68876562e-01, 8.25253758e-01],
       [1.22809050e-01, 1.16491778e-01, 8.17082125e-01, 9.00507979e-01,
        3.95363000e-01, 1.97203518e-01, 8.88472858e-01, 8.60260442e-01,
        2.09191870e-01, 8.01976850e-01, 1.64229029e-01, 4.13945091e-01,
        4.10994382e-01, 4.35582063e-01, 9.70852976e-01, 2.21240297e-01,
        2.20131275e-01, 3.90878510e-01, 1.90026477e-01, 1.64982775e-01,
        8.78797455e-02, 7.17127718e-01, 1.19098413e-01, 5.10003788e-01,
        9.98933082e-01, 4.67415217e-01, 4.70478803e-01, 7.96226

In [12]:
import sys, os
sys.path.append(os.pardir)
from common.util import im2col

x1=np.random.rand(1,3,7,7) # 데이터, 채널, 높이, 넓이
col1=im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

x2=np.random.rand(10,3,7,7)
col2=im2col(x2,5,5,stride=1,pad=0)
print(col2.shape)

(9, 75)
(90, 75)


In [14]:
# convolution 계층
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W=W
        self.b=b
        self.stride=stride
        self.pad=pad

    def forward(self, x):
        FN, C, FH, FW=self.W.shape
        N,C,H,W=x.shape
        out_h=int(1+(H+2*self.pad-FH)/self.stride)
        out_w=int(1+(W+2*self.pad-FW)/self.stride)

        col=im2col(x,FH,FW, self.stride, self.pad)
        col_W=self.W.reshape(FN, -1).T
        out=np.dot(col, col_W) + self.b

        out=out.reshape(N,out_h,out_w, -1).transpose(0,3,1,2)

        return out

In [18]:
# 풀링 계층
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad

    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)

        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h * self.pool_w)

        out = np.max(col, axis=1)

        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

        return out

In [20]:
class SimpleConvNet:
    def __init__(self, input_dim=(1, 28, 28),
                 conv_param={'filter_num': 30, 'filter_size': 5,
                             'pad': 0, 'stride': 1},
                 hidden_size=100, output_size=10, weight_init_std=0.01):
        # 초기화 인수로 주어진 하이퍼파라미터를 딕셔너리에서 꺼내고 출력 크기를 계산한다.
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))

        # 1층의 합성곱 계층과 2, 3층의 완전연결 계층의 가중치와 편향 생성
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)

        # CNN을 구성하는 계층을 생성
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'],
                                           self.params['b1'],
                                           conv_param['stride'],
                                           conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
        self.last_layer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x

    def loss(self, x, t):
        # 손실함수
        y = self.predict(x)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1:
            t = np.argmax(t, axis=1)

        acc = 0.0

        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt)

        return acc / x.shape[0]

    def gradient(self, x, t):
        # 오차역전파법
        # 순전파
        self.loss(x, t)

        # 역전파
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'] = self.layers['Conv1'].dW
        grads['b1'] = self.layers['Conv1'].db
        grads['W2'] = self.layers['Affine1'].dW
        grads['b2'] = self.layers['Affine1'].db
        grads['W3'] = self.layers['Affine2'].dW
        grads['b3'] = self.layers['Affine2'].db

        return grads